## CountV LAC 2017 (concise case)

modelling concise case of LAC 2017 use Count Vector as word embedding

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from gensim.models import Word2Vec
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

C:\Users\Aska\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# for model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import model_from_json
from keras.callbacks import CSVLogger


Using TensorFlow backend.


In [3]:
DATA_PATH="../Data/Raw/"
DATA_PATH2="../..//Data/Manipulation/"
MODEL_PATH="../../Models/"

### CountV Vectorize

In [4]:
dta_tr=pd.read_csv(DATA_PATH2+"data_train_tokenize_concise_only2_train.csv")
dta_te=pd.read_csv(DATA_PATH2+"data_train_tokenize_concise_only2_test.csv")

In [5]:
dta_tr["true_text"]=dta_tr.token_short_des.apply(lambda x: " ".join(ast.literal_eval(x)))
dta_te["true_text"]=dta_te.token_short_des.apply(lambda x: " ".join(ast.literal_eval(x)))

In [6]:
docs=dta_tr.true_text.tolist()
docs_te=dta_te.true_text.tolist()

In [7]:
docs[:2]

['formulated oil free hydrating botanicals remarkably improves skin texture abused hands restores soft smooth refined hands',
 '150cm mini microphone compatible iphone various smartphones also ipad apple computer macbook dual headed design allows two people using simultaneously features high sensitivity omni directional sounds output perfect audio video recording 3 5mm standard connector jack convenient clip design clip collar 3 5mm standard connector jack convenient clip design clip collar']

In [8]:
%%time
countv_trans_75=CountVectorizer(max_features=75)
countv_trans_75.fit(docs)

Wall time: 1.34 s


In [9]:
%%time
countv_trans_150=CountVectorizer(max_features=150)
countv_trans_150.fit(docs)

Wall time: 1.31 s


In [10]:
%%time
countv_trans_225=CountVectorizer(max_features=225)
countv_trans_225.fit(docs)

Wall time: 1.32 s


In [11]:
A=countv_trans_75.transform([docs[0]])

In [12]:
A.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [13]:
joblib.dump(countv_trans_75,MODEL_PATH+"countv75_transform.pkl")
joblib.dump(countv_trans_150,MODEL_PATH+"countv150_transform.pkl")
joblib.dump(countv_trans_225,MODEL_PATH+"countv225_transform.pkl")

['../../Models/countv225_transform.pkl']

### Modeling Part

In [41]:
EPOCH=8
BATCH=595

In [42]:
def cur_model(input_size):
    model = Sequential()
#     model.add(Embedding(50, 12, input_length=input_size))
    
#     Embedding(vocab_size, embd2.layer1_size, weights=[embedding_vectors], 
#                             input_length=max_length, trainable=False)
    
#     model.add(Conv1D(filters=8, kernel_size=5, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
#     model.add(Flatten())
    model.add(Dense(200,input_dim=input_size, activation='relu'))
    model.add(Dense(200,input_dim=input_size, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(70, activation='relu'))
    model.add(Dense(70, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [43]:
Y_true=dta_tr["conci"]

In [44]:
X75=countv_trans_75.transform(dta_tr.true_text)
X150=countv_trans_150.transform(dta_tr.true_text)
X225=countv_trans_225.transform(dta_tr.true_text)

In [45]:
# define model 75
model75=cur_model(75)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 200)               15200     
_________________________________________________________________
dense_44 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_13 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 70)                14070     
_________________________________________________________________
dense_46 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_14 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 15)                1065      
__________

In [46]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'countv_log75.csv', append=True, separator=',')
model75.fit(X75, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

Wall time: 18.1 s


In [47]:
# define model 150
model150=cur_model(150)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 200)               30200     
_________________________________________________________________
dense_51 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_15 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 70)                14070     
_________________________________________________________________
dense_53 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_16 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 15)                1065      
__________

In [48]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'countv_log150.csv', append=True, separator=',')
model150.fit(X150, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

Wall time: 12.7 s


In [49]:
# define model 225
model225=cur_model(225)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 200)               45200     
_________________________________________________________________
dense_58 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_17 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 70)                14070     
_________________________________________________________________
dense_60 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_18 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 15)                1065      
__________

In [50]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'countv_log225.csv', append=True, separator=',')
model225.fit(X225, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

Wall time: 22.7 s


In [51]:
# Model Saving

# serialize model to JSON
for model,mod_name in [(model75,"countv_model75"),(model150,"countv_model150"),(model225,"countv_model225")]:
    model_json = model.to_json()
    with open(MODEL_PATH+mod_name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(MODEL_PATH+mod_name+"_weight.h5")
    print("Saved model to disk")

Saved model to disk
Saved model to disk
Saved model to disk


### Evaluate

In [52]:
for model,X in [(model75,X75),(model150,X150),(model225,X225)]:
    loss, acc = model.evaluate(X, Y_true, verbose=0)
    print('Train Accuracy: %f' % (acc*100))

Train Accuracy: 72.634244
Train Accuracy: 74.195227
Train Accuracy: 75.992091


In [53]:
X75_te=countv_trans_75.transform(dta_te.true_text)
X150_te=countv_trans_150.transform(dta_te.true_text)
X225_te=countv_trans_225.transform(dta_te.true_text)

Y_test=dta_te.conci

In [54]:
for model,X in [(model75,X75_te),(model150,X150_te),(model225,X225_te)]:
    loss, acc = model.evaluate(X, Y_test, verbose=0)
    print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 71.859155
Test Accuracy: 72.408451
Test Accuracy: 73.591549
